<a href="https://colab.research.google.com/github/aecins/tutorials/blob/main/rerun_transforms_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install ReRun. Runtime may needs to be restarted after rerun is installed.
try:
  import rerun as rr
except ImportError:
  !pip3 install rerun-sdk[notebook]
  print('Stopping RUNTIME! Please run again.')
  import os
  os.kill(os.getpid(), 9)

In [6]:
# Install library for handling 3D transforms
!pip3 install spatialmath-python

In [7]:
import numpy as np
from spatialmath import SE3, SO3

WORLD_ENTITY_FRAME = '/world'
ROBOT_ENTITY_FRAME = '/world/robot'
SENSOR_ENTITY_FRAME = '/world/robot/sensor'

# World frame is static
root_T_world = SE3()

# Robot moves between two poses with constant motion.
world_T_robot__start = SE3()
axis = np.array([1, 1, 1])
# angle_end = np.pi
angle_end = 0
translation_end = np.array([10, 0, 0])
num_steps = 10

world_T_robot = {
    'transforms': [],
    'timestamps': [],
    'entity_path': ROBOT_ENTITY_FRAME,
}

for idx in range(num_steps):
    timestamp = idx
    translation = translation_end / num_steps * idx
    angle = angle_end / num_steps * idx
    rotation = SO3.AngleAxis(theta=angle, v=axis)

    world_T_robot__cur = SE3.Rt(R=rotation, t=translation)

    world_T_robot['transforms'].append(world_T_robot__cur)
    world_T_robot['timestamps'].append(timestamp)

# A sensor is rigidly attached to the robot.
robot_tx_sensor__static = [0, 0, 1]
robot_R_sensor__static = SO3.Rx(theta=np.pi / 2.0)
robot_T_sensor__static = SE3.Rt(R=robot_R_sensor__static, t=robot_tx_sensor__static)

# A sensor non-rigidly attached to the robot
robot_T_sensor__dynamic = {
    'transforms': [SE3.Rt(R=SO3(), t=[0, 1, 0]), SE3.Rt(R=SO3(), t=[0, 5, 0])],
    'timestamps': [2.5, 5.5],
    'entity_path': SENSOR_ENTITY_FRAME,
}

In [8]:
rr.init("rerun_viewer")

def log_static_transform(entity_frame, transform_se3):
    rr.log(
        entity_frame,
        rr.Transform3D(mat3x3=transform_se3.R, translation=transform_se3.t),
        static=True,
    )


def log_dynamic_transform(transform):
    for idx in range(len(transform['transforms'])):
        parent_T_child = transform['transforms'][idx]
        timestamp = transform['timestamps'][idx]

        # Set timestamp.
        rr.set_time_seconds("hardware_timestamp", timestamp)

        # Set transform.
        rr.log(
            transform['entity_path'],
            rr.Transform3D(mat3x3=parent_T_child.R, translation=parent_T_child.t),
            static=False,
        )


# Log origin transform
log_static_transform(WORLD_ENTITY_FRAME, root_T_world)

# Log robot pose
log_dynamic_transform(world_T_robot)

# Log sensor extrinsics
# log_static_transform(SENSOR_ENTITY_FRAME, robot_T_sensor__static)
log_dynamic_transform(robot_T_sensor__dynamic)

In [9]:
rr.notebook_show(width=1400, height=800)

Viewer()

### Takeaways
* when logging a transform using `rr.log(entity_path='/world/robot/sensor', ...)` defines a transform with:
  * parent is `/world/robot`
  * child is `/world/robot/sensor`
* static transforms move with the parent frame. I.e. if parent frame moves relative to the origin frame - the child transform statically attached to the parent frame will "move together" with the parent frame
* transforms are not interpolated, instead the last avilable transform is used. Consider a robot `baselink` frame moving between two different poses at timestamps `t=0` and `t=1`. ReRun will render the pose of the frame as follows:
  * for $t \in [0,1)$ - `baselink` frame will be visualized using pose at `t=0`
  * for $t \in [1, +\inf)$ - `baselink` frame will be visualized using pose at `t=1`